# Extract Audio Feautures


In [1]:
import sys
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences


import audiofile
import opensmile
from util import *


In [2]:
pd.set_option('display.max_columns', None)


In [3]:
def configure_openSmile(**kwargs):
    '''
    Configure OpenSmile.

        Parameters:
            folder_list (list): A list of folder paths to extract features

        Returns:
            smile (obj): Returns configured smile object
    '''
    
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
        loglevel=4,
        logfile='smile.log',
        verbose=True,

    )
    return smile


In [4]:
def extract_feautre(folder_list):
    '''
    Extracts audio feature for each file in file path.

        Parameters:
            folder_list (list): A list of filepaths to extract audio features

        Returns:
            features (Dataframe): Extracted features from USC-Iemocap database
    '''
    
    smile = configure_openSmile()
    try:
        features = smile.process_files(folder_list)
        return features

    except FileNotFoundError as e:
        print(f'Error --> {e}')


In [5]:
def z_score(df, inplace=False):
    '''
    Apply z-score (standardization) to the columns of the database.

        Parameters:
            df (DataFrame): DataFrame to apply z-score
            inplace: Modify given DataFrame

        Returns:
            df (DataFrame): DataFrame with applied z-score
    '''

    if inplace:
        df = df.copy()
        
    # apply the z-score method
    for column in df.columns:
        df[column] = (df[column] - df[column].mean()
                          ) / df[column].std()

    print('Finished standardizing (z-score) data')
    return df


In [20]:
def pad_signal_data(signal_sequence, padding='pre'):
    '''
    Pad each signal with zeros for eqaul data dimenstions.

        Parameters:
            signal_sequence (List): Signal list to be padded
            padding (string):  ('pre' or 'post') pad either before or after each sequence

        Returns:
            padded (DataFrame): Padded dataframe
    '''

    # for count, file in enumerate(files_list):
    #     data_frame = df.loc[file] # Get extracted file feature from DataFrame
        
    #     # Max num of signal(rows) extracted is 3409, extend frame by 3409 - len signal
    #     no_extend_row = 3409 - len(df.loc[file])

    #     arr = pd.DataFrame(
    #         data=np.zeros((no_extend_row, 65)), \
    #         columns=df.columns) # Create dataframe with zeros

    #     # Append to dataframe
    #     df.loc[file] = pd.concat([data_frame, arr])
    #     # data_frame.appende(arr, ignore_index=True)

    padded = pad_sequences(signal_sequence, dtype='float64', padding=padding)

        # if count % 1000 == 0:
        #     print(f'Count{count} -- {len(files_list)}')
        #     sys.stdout.flush()

    print('Finished padding data')
    return padded


In [10]:
full_ids = []
with open('./data/processed/IEMOCAP/processed_ids.txt') as f:
    full_ids = f.readlines()
full_ids = [x.strip() for x in full_ids]


In [11]:
list_files = []

for x in range(1, 6):
    sess_title = 'Session' + str(x)

    path = f'./data/raw/IEMOCAP_full_release/{sess_title}/sentences/wav'

    file_search(path, list_files)
    list_files = sorted(list_files)

    print(f"{sess_title}, # Num of files: {len(list_files)}")


Session1, # Num of files: 1819
Session2, # Num of files: 3630
Session3, # Num of files: 5766
Session4, # Num of files: 7869
Session5, # Num of files: 10039


## Get list of filenames in category, (Angry, Happy, Sad, Neutral)

In [12]:
full_path = []

for file_ in full_ids:
    for file2 in list_files:
        if file_+'.wav' == file2.split('\\')[-1]:
            full_path.append(file2)
            break


In [ ]:
print(full_path)

## Extract four category feature

In [ ]:
# features = extract_feautre(full_path)
# features.to_pickle('D:/feat.pk')

## Post-processing: Load Pickled feature

In [15]:
with open('D:/feat.pk', 'rb') as pickle_file:
    data = pickle.load(pickle_file)


In [53]:
feature = []

for idx, file in enumerate(full_path):
    feature.append(data.loc[file].values.tolist())
    # feature.append(data.loc[file].to_numpy())

print(len(feature))


5531


In [55]:
padded_data = pad_signal_data(feature, padding='post')

MemoryError: Unable to allocate 4.57 GiB for an array with shape (5531, 3409, 65) and data type int32

In [26]:
print(padded_data.shape)

(5531, 3409, 65)


In [35]:
padded_data[0][0]

array([     0,      0,      0,      0,      0,   -100,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      0,      0,      0,      0,      0,    197,    375,
          447,   1125,      0,    553,      4, 595613,      3,     16,
            0,      0,      0,     21,     -4,     16,    -16,     11,
          -12,     10,    -20,     -1,      4,     18,      3,     20,
            7])

In [ ]:
full_cat_label = []
with open('./data/processed/IEMOCAP/processed_label.txt') as f:
    full_cat_label = f.readlines()
full_cat_label = [x.strip() for x in full_cat_label]


## lll

In [13]:
label_data = []
with open('./data/processed/IEMOCAP/processed_label.txt') as f:
    label_data = f.readlines()

with open('./data/processed/IEMOCAP/FC_label.txt', 'w') as f:
    for i, label in enumerate(full_cat_label):
        if label != '-1':
            f.write(label_data[i])


In [ ]:
data_norm = z_score(padded_data)
save_as_pickle('D:/feature_z_score.pkl', data_norm)


In [ ]:
with open('list_feat.pkl', 'wb') as f:
    pickle.dump(feature, f)


## Save numpy values in file

In [ ]:
# np.save('D:/Features.npy', data.values)